In [1]:
import pandas as pd
import numpy as np
import math

import os
from tqdm import tqdm
import json
import shutil

# image data
from PIL import Image
# import cv2

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from glob import glob
import matplotlib.pyplot as plt

In [3]:
!nvidia-smi

Mon Oct 31 07:55:03 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:00:05.0 Off |                  Off |
| N/A   49C    P0   154W / 300W |  31134MiB / 32480MiB |     98%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:00:06.0 Off |                  Off |
| N/A   33C    P0    64W / 300W |    317MiB / 32480MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
        

### DCM header check

In [3]:
# dcm header check
import pydicom

path = '/home/ncp/workspace/202002n035/035.신장암 진단을 위한 의료 영상 데이터/01.데이터/신장암2/1.Training/원천데이터/'
path__ = path+'02_01_0007/02_01_0007_PRE/0174.dcm'
header_01 = pydicom.dcmread(path__, stop_before_pixels = True)
header_01.InstanceNumber

'82'

In [2]:
import pydicom

In [ ]:
df = pd.DataFrame(columns = ['name', 'modality'])
data = [{'name' : '0', 'modality': str(header_01[0x0008, 0x0001030][0:])}]
tmp = pd.DataFrame(data)
tmp

In [ ]:
# dcm header별 modality list
path = '/home/ncp/workspace/202002n035/035.신장암 진단을 위한 의료 영상 데이터/01.데이터/신장암2/1.Training/원천데이터/'
patient = os.listdir(path)
df = pd.DataFrame(columns = ['name', 'modality'])
lst = []

for ppl in patient: 
    deep_path = path + ppl + '/' + '{}_ART'.format(ppl) + '/'
    filelist = os.listdir(deep_path)
    for file in filelist: 
        try: 
            header = pydicom.dcmread(deep_path + file, stop_before_pixels = True)
            data = [{'name' : ppl, 'modality': str(header[0x0008, 0x0001030][0:])}]
            tmp = pd.DataFrame(data)
            df = pd.concat([df, tmp], axis = 0)
        except Exception: 
            lst.append(ppl)
            pass

In [ ]:
df

In [ ]:
df['modality'].unique()

In [ ]:
df_groupby = df.groupby(['modality']).count().reset_index()
df_groupby

In [ ]:
set(lst)

In [ ]:
modality_kidney_list = df.loc[df['modality'].isin(['CT Kidney (3P) + 3D', 'CT Kidney (3P) + 3D (contrast)']) == True, 'name'].unique()
len(modality_kidney_list)

In [ ]:
modality_kidney_list

In [ ]:
modality_pelvis_list = df.loc[df['modality'].isin(['Pelvis^00_Kidney_3D (Adult)']) == True, 'name'].unique()
len(modality_pelvis_list)

In [ ]:
# 사람별 ct 이미지 장수 확인

n_list = []

for ppl in modality_kidney_list: 
    lst = glob('/home/ncp/workspace/blocks1/kidneyData_windowing/TRAIN/ART/{}_*'.format(ppl))
    lst.sort()
    n_list.append(len(lst))
    print(ppl, "N: ", len(lst))

In [ ]:
# 사람별 ct 이미지 장수 확인

n_list = []

for ppl in modality_pelvis_list: 
    lst = glob('/home/ncp/workspace/blocks1/kidneyData_windowing/TRAIN/ART/{}_*'.format(ppl))
    lst.sort()
    n_list.append(len(lst))
    print(ppl, "N: ", len(lst))

In [ ]:
n_list = pd.DataFrame(n_list)

In [ ]:
n_list[0].describe()

In [ ]:
# 0장인 사람이?
# 0drop후 describe

n_list = n_list.loc[n_list[0] != 0].reset_index(drop = True)
n_list[0].describe()